In [101]:
from mongoengine import *
import numpy as np
import datetime 
import pandas
import matplotlib.pyplot as plt
import sklearn
import tabulate

In [9]:
connect("GDP-test", host = "localhost", port = 27017)
class Detection(Document):
    id_webcam = IntField(required=True)
    city = StringField(required=True)
    location = StringField(required=True)
    latitude = FloatField(required=True)
    longitude = FloatField(required=True)
    numPeople = IntField(required=True)
    date = DateTimeField(required=True)
    time = DateTimeField(required=True)
    type = IntField(required=True)
    weather_description = StringField()
    temperature = FloatField()
    day_of_week = IntField()

In [125]:
#print(Detection.objects.distinct('city')) # vedere se posso prendere solo le città per poi fare query distinte
table = pandas.DataFrame(Detection.objects(city='Roma').as_pymongo()) #prelevo i dati di roma

most_freq = lambda x: x.value_counts(dropna=False).index[0]

table_a = table
table_a = table_a.drop(columns = ['_id','id_webcam','city','type','date','location','latitude','longitude'])
table_a['time'] = pandas.to_datetime(table_a['time'])
table_a.sort_values(by='time', inplace=True)
print(table_a.columns)
#table_1 = table_a.resample('30T', on='time').agg({'numPeople': 'mean', 'temperature' : 'mean', 'weather_description'}).head(20)
tb = table_a.groupby([pandas.Grouper(key='time', freq='30T')]).agg(  meanPeople=('numPeople', 'mean')
                                                                   , temp=('temperature','mean')
                                                                   , weather=('weather_description', most_freq )
                                                                   , day_of_week=('day_of_week', most_freq))

# tb.plot('time','meanPeople')
# tb['hour'] = tb['time'].apply()
# #print(type tb['time'])
print(tb.to_markdown())
print(table_a.to_markdown())
# tb.plot('time','numPeople')

Index(['numPeople', 'time', 'weather_description', 'temperature',
       'day_of_week'],
      dtype='object')
| time                |   meanPeople |      temp | weather          |   day_of_week |
|:--------------------|-------------:|----------:|:-----------------|--------------:|
| 2021-03-24 09:00:00 |      7       |   7.675   | clear sky        |             2 |
| 2021-03-24 09:30:00 |      9.83333 |   9.24833 | clear sky        |             2 |
| 2021-03-24 10:00:00 |     10.1667  |  10.35    | clear sky        |             2 |
| 2021-03-24 10:30:00 |     14.5     |  11.55    | clear sky        |             2 |
| 2021-03-24 11:00:00 |     15.1667  |  12.8167  | clear sky        |             2 |
| 2021-03-24 11:30:00 |     13       |  13.5667  | clear sky        |             2 |
| 2021-03-24 12:00:00 |     13.5     |  14.555   | clear sky        |             2 |
| 2021-03-24 12:30:00 |     11.5     |  14.96    | clear sky        |             2 |
| 2021-03-24 13:00:00 |     1

In [122]:
print(tb.columns)
#tb.plot('meanPeople','time')

Index(['meanPeople', 'temp', 'weather', 'day_of_week'], dtype='object')


In [104]:
tb['time'] = tb['time'].dt.strftime('%H').astype(int)#modifico le colonne con i valori che mi servono
table['weather_description'] = pandas.get_dummies(table['weather_description']) #il weather viene convertito con parametri interi

#divide the dataframe 70-30 for train and test
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
    table[['time','temperature','day_of_week','weather_description']], 
    table['numPeople'], 
    test_size = 0.33, shuffle = True, random_state= 42)

KeyError: 'time'

In [99]:
table = table.drop(columns = ['_id','id_webcam','city','date','type','location','latitude','longitude']) #tolgo colonne inutili
table.plot('time','numPeople')
table['time'] = table['time'].dt.strftime('%H').astype(int)#modifico le colonne con i valori che mi servono
table['weather_description'] = pandas.get_dummies(table['weather_description']) #il weather viene convertito con parametri interi
#print(table.to_markdown())

start = datetime.datetime(2021,3,26)
end = datetime.datetime(2021,3,27)
raw_query = {'date': {'$gte': start, '$lt': end}, 'city' : 'Roma'}
table = pandas.DataFrame(Detection.objects(__raw__=raw_query).as_pymongo())
table = table.drop(columns = ['_id','id_webcam','city','date','type','location','latitude','longitude'])
table.plot('time','numPeople')

#divide the dataframe 70-30 for train and test
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
    table[['time','temperature','day_of_week','weather_description']], 
    table['numPeople'], 
    test_size = 0.33, shuffle = True, random_state= 42)


KeyError: "['_id' 'id_webcam' 'city' 'date' 'type' 'location' 'latitude' 'longitude'] not found in axis"

In [ ]:
model = sklearn.linear_model.LinearRegression()
#provo a dargli in pasto tutto
#prima di questo ora bisogna dividere tutto il dataset in 70-30
model.fit(x_train, y_train)


# The coefficients
print('Coefficients: \n', model.coef_)

# The mean square error
print("Residual sum of squares: %.2f" % np.mean((model.predict(x_test) - y_test) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % model.score(x_test, y_test))

plt.plot(x_test['time'],y_test)


In [ ]:

#robe vecchie sbagliate non guardare

time, temperature, weather, numP = table['time'].dt.strftime('%H').astype(int),table['temperature'],table['weather_description'],table['numPeople']
time = np.asarray(time)
temperature = np.asarray(temperature)
weather = np.asarray(weather)
numP = np.asarray(numP)
weatherDict = {'clear sky':0}

In [ ]:
for i in range(weather.size):
    weather[i] = weatherDict[weather[i]]

In [ ]:
temperature = temperature.reshape(-1,1)
time = time.reshape(-1,1)
weather = weather.reshape(-1,1)
timeTemp = np.concatenate((time,temperature,weather),axis = 1)
print(timeTemp)
model = sklearn.linear_model.LinearRegression()
model.fit(timeTemp,numP) 

In [ ]:
start = datetime.datetime(2021,3,24)
end = datetime.datetime(2021,3,29)
raw_query = {'date': {'$gte': start, '$lt': end}, 'city' : 'Roma'}
table = pandas.DataFrame(Detection.objects(__raw__=raw_query).as_pymongo())
table = table.drop(columns = ['_id','id_webcam','city','date','type','location','latitude','longitude'])